In [ ]:
"""
Created on Mon Oct 21 16:13 2024

Prepare what's needed for eORCA1 on the ice-shelf side

@author: Clara Burgard
"""

In [1]:
import xarray as xr
import numpy as np
from cdo import Cdo
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from datetime import date

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

this is CDO version 2.1.0


In [4]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

REMAP THE ICE SHELF CONCENTRATION

In [5]:
fNEMO_mask = 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc'
dom_cfg_path = inputpath_raw + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc'
mask_file_path = inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc'
grid_file_path = inputpath_interim + 'grid_T_eORCA1_withoutcellarea.txt'
grid_file_path2 = inputpath_interim + 'NEMO_gridT_eORCA1_cdo.nc'
grid_file_path3 = '/data/cburgard/CASIMIR_SIMU/raw/n42openc/n42openc_00910101_01001231_1Y_grid_T.nc'

#grid_file_path = inputpath_raw + 'eORCA1_v4.2_CDO_with_var.nc'

In [ ]:
cdo.selvar('isf_conc', input = mask_file_path, output = inputpath_interim + 'isf_conc_only_2km.nc')

In [ ]:
cdo.remapcon(grid_file_path3, input = inputpath_interim + 'isf_conc_only_2km_setgrid.nc', output = inputpath_interim + 'isf_conc_only_targetgrid.nc')

REMAP THE IMBIE MASK

In [ ]:
cdo.selvar('Iceshelf_extrap', input = mask_file_path, output = inputpath_interim + 'isfmask_only_2km.nc')

In [ ]:
# took a grid from Casimir
cdo.remapnn(grid_file_path, input = inputpath_interim + 'isfmask_only_2km.nc', output = inputpath_interim + 'isfmask_only_targetgrid.nc')

!!!!!!! ONLY IF YOUR GRID IS eORCA1 !!!!!!!

In [ ]:
# For eORCA1, this method leads to no ocean cells for Bach (ID=101) and Cosgrove (ID=88). I suggest merging it with its neighbour 'Britten' (ID=100), and neighbour 'No Name' (ID=89) which has no melt in most of the obs products
isfmask = xr.open_dataset(inputpath_interim + 'isfmask_only_targetgrid.nc')
isfmask = isfmask.where(isfmask != 100, 101)
isfmask = isfmask.where(isfmask != 89, 88)

CREATE MASK_VARIABLE

In [ ]:
domain_cfg = xr.open_dataset(dom_cfg_path)
isfconc = xr.open_dataset(inputpath_interim + 'isf_conc_only_targetgrid.nc')['isf_conc']

In [ ]:
isfconc.plot()

In [18]:
isfconc_orig = xr.open_dataset(inputpath_interim + 'isf_conc_only_2km_setgridIMBIE.nc')

In [16]:
3041*3041

9247681

In [19]:
isfconc_orig

<xarray.Dataset>
Dimensions:   (x: 3041, y: 3041)
Coordinates:
  * x         (x) float64 -3.04e+06 -3.038e+06 -3.036e+06 ... 3.038e+06 3.04e+06
  * y         (y) float64 -3.04e+06 -3.038e+06 -3.036e+06 ... 3.038e+06 3.04e+06
Data variables:
    isf_conc  (y, x) float64 ...
Attributes:
    CDI:          Climate Data Interface version 2.1.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed Oct 30 17:17:55 2024: cdo setgrid,IMBIE_2km_stereo_with...
    CDO:          Climate Data Operators version 2.1.0 (https://mpimet.mpg.de...

In [15]:
isfconc_orig

<xarray.Dataset>
Dimensions:   (y: 3041, x: 3041)
Coordinates:
    lon       (y, x) float64 ...
    lat       (y, x) float64 ...
  * x         (x) float64 -3.04e+06 -3.038e+06 -3.036e+06 ... 3.038e+06 3.04e+06
  * y         (y) float64 -3.04e+06 -3.038e+06 -3.036e+06 ... 3.038e+06 3.04e+06
Data variables:
    isf_conc  (y, x) float64 ...
Attributes:
    CDI:          Climate Data Interface version 2.1.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Wed Oct 30 12:16:50 2024: cdo -O -s -selvar,isf_conc /data/...
    CDO:          Climate Data Operators version 2.1.0 (https://mpimet.mpg.de...

In [ ]:
grid_file = xr.open_dataset(inputpath_interim + 'NEMO_gridT_eORCA1_cdo.nc')

In [ ]:
grid_file

In [ ]:
grid_file['bounds_nav_lat'].isel(nvertex=1).plot()

In [ ]:
isfconc_orig['isf_conc'].plot()

In [ ]:
ds_isfNEMO = xr.Dataset()

ds_isfNEMO['mask_isf'] = isfmask['Iceshelf_extrap'].where(domain_cfg['isf_draft'] > 0)
ds_isfNEMO['mask_isf'].attrs['standard_name'] = 'mask of all the ice shelves'
ds_isfNEMO['mask_isf'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

#ds_isfNEMO['mask_isf_onlyisfconc1'] = ds_isfNEMO['mask_isf'].where(isfconc['isf_conc'] > 0.9999)


In [ ]:
ds_isfNEMO['floating_frac'] = isfconc['isf_conc'] # .where(isfconc['isf_conc'] < 0.9999,1)

CREATE MASK_ISF_OPEN AND MASK_ISF_CLOSED

In [ ]:
file_Justine = xr.open_dataset(mask_file_path).drop('quantile')

In [ ]:
## gives the list of names when needed to better understand stuff
#for iid in file_Justine.ID:
#    print(iid.values,file_Justine['NAME'].sel(ID=iid).values)

In [ ]:
# Choose (by eye or given a criterion, which ice shelves will have open cavities)
ID_open_list = [1,2,21,66,67,117,124,125,127,128]

In [ ]:
# Divide the ref file into "open" and "closed" cavities
file_Justine_open = file_Justine.sel(ID=ID_open_list)
file_Justine_closed = file_Justine.drop_sel(ID=ID_open_list)

In [ ]:
domain_cfg_closed['isf_draft'].plot()

In [ ]:
# Prepare domain_cfg closing (setting to 0) all information about the closed cavities
domain_cfg_closed =  domain_cfg.copy()
for id_closed in tqdm(file_Justine_closed.ID):
    for vv in ['bottom_level','top_level','isf_draft','bathy_metry']:
        domain_cfg_closed[vv] = domain_cfg_closed[vv].where(ds_isfNEMO['mask_isf'].drop_vars(['glamt', 'gphit']) != id_closed.values, 0)

In [ ]:
# Prepare mask of only open cavities 
mask_isf_open = ds_isfNEMO['mask_isf'].copy()
for id_closed in tqdm(file_Justine_closed.ID):
    mask_isf_open = mask_isf_open.where(ds_isfNEMO['mask_isf'].drop_vars(['glamt', 'gphit']) != id_closed.values)

In [ ]:
# Prepare mask of only closed cavities 
mask_isf_closed = ds_isfNEMO['mask_isf'].where(np.isnan(mask_isf_open))

In [ ]:
# create mask_isf_open and mask_isf_closed
ds_isfNEMO['mask_isf_open'] = mask_isf_open
ds_isfNEMO['mask_isf_open'].attrs['standard_name'] = 'mask of the ice shelves that are open'
ds_isfNEMO['mask_isf_open'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

ds_isfNEMO['mask_isf_closed'] = mask_isf_closed
ds_isfNEMO['mask_isf_closed'].attrs['standard_name'] = 'mask of the ice shelves that are closed'
ds_isfNEMO['mask_isf_closed'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

IDENTIFY FRONT

In [ ]:
# Create a mask discriminating between land, ocean and ice shelf
mask_0_1_2 = ds_isfNEMO['mask_isf'] * 2
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] != 0, 400) # land
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] == 0, 0) # ocean
mask_0_1_2 = mask_0_1_2.where(domain_cfg['isf_draft'] == 0, 200) # ice shelf

In [ ]:
# Create a mask of the extrapolated domains and only keep the domains corresponding to the closed cavities
extrap_domains_closed = isfmask['Iceshelf_extrap'].copy()

for kisf in ID_open_list:
    mask_0_1_2_closed = mask_0_1_2.where(ds_isfNEMO['mask_isf'] != kisf, 400)
    extrap_domains_closed = extrap_domains_closed.where(extrap_domains_closed != kisf, np.nan)

In [ ]:
# set all ice shelves to 300
mask_front0 = mask_0_1_2.where((mask_0_1_2 == 0) | (mask_0_1_2 == 400), 300).copy()

In [ ]:
mask_front = mask_front0.copy()
# check all directions and set points at border between ocean and ice shelf (300-0) to 500
mask_front = mask_front.where((mask_front0.shift(x=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(x=1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(y=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(y=1)-mask_front0)!=300,500)
# cut out all front points
mask_front = mask_front.where(mask_front==500)
# set the ice shelf number
mask_front = mask_front.where(mask_front!=500,extrap_domains_closed)

For the ice shelves not resolved at all in eORCA1

In [ ]:
# define the domains that have not an associated front yet
extrap_domains_double_closed = extrap_domains_closed.copy()

for kisf in file_Justine_closed.ID:
    if (mask_front == kisf).astype(int).sum() > 0:
        #print(kisf.values)
        extrap_domains_double_closed = extrap_domains_double_closed.where(extrap_domains_double_closed != kisf, np.nan).drop('ID')

In [ ]:
# To double check where the front points are
#iid = 89
#plt.figure()
#((mask_0_2) + (isfmask['Iceshelf_extrap'] == iid)).plot()
#plt.title(str(iid))

In [ ]:
# set all ice shelves to land to have a delimitation of the whole contour of Antarctica
mask_0_2 = mask_0_1_2.where(mask_0_1_2 != 200, 2)
mask_0_2 = mask_0_2.where(mask_0_2 != 400, 2)

mask_front_new = mask_0_2.copy()

# check all directions and set points at border between ocean and land(300-0) to 500
mask_front_new = mask_front_new.where((mask_0_2.shift(x=-1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(x=1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(y=-1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(y=1)-mask_0_2) <= 0,5)
# cut out all front points south of the y = 90 latitude and in the ocean domain
mask_front_new = mask_front_new.where((mask_front_new == 5) & (mask_front_new.y < 90) & (mask_0_2 == 0))
# set the ice shelf number of the extrapolated domain
mask_front_new = mask_front_new.where(mask_front_new != 5,extrap_domains_double_closed)

In [ ]:
# check that it is the right delimitation (i.e. on the ocean points)
((mask_front_new > 0) + mask_0_2).plot()

In [ ]:
# combining the masks of the fronts (mask_front and mask_front_new)
mask_front_all = mask_front.copy()
for iid in file_Justine_closed.ID:
    if (ds_isfNEMO['mask_isf_closed'] == iid).astype(int).sum().values == 0: # if the ice shelf is not resolved in your grid, add the mask front inferred for them
        #print(iid.values, file_Justine_closed['NAME'].sel(ID=iid).values, (mask_front_new == iid).astype(int).sum().values)
        mask_front_all = mask_front_all.combine_first(mask_front_new.where(mask_front_new == iid))

MAKE MAPS OF ZMIN AND ZMAX ON THE FRONT

In [ ]:
zmin_front = mask_front_all.copy()
zmax_front = mask_front_all.copy()

for id_closed in tqdm(file_Justine_closed.ID):
    zmin_front = zmin_front.where(mask_front_all != id_closed, file_Justine_closed['z_min'].sel(ID=id_closed)).drop('ID')
    zmax_front = zmax_front.where(mask_front_all != id_closed, file_Justine_closed['z_max'].sel(ID=id_closed)).drop('ID')

check if the extremes or the percentiles are more "realistic"

In [ ]:
file_Justine_closed['z_min'].plot()
file_Justine_closed['z_perc01'].plot()

In [ ]:
file_Justine_closed['z_max'].plot()
file_Justine_closed['z_perc99'].plot()

DISTRIBUTE MELT AT THE FRONT NORMALISED BY CELL SIZE

In [ ]:
cell_area = domain_cfg['e1t'] * domain_cfg['e2t']
melt_src = 'Adusumilli' # options: 'Adusumilli','Rignot', 'Paolo', 'Davison'
melt_flux = file_Justine_closed['Melt'+melt_src]

In [ ]:
melt_front = mask_front_all.copy()
for id_closed in tqdm(file_Justine_closed.ID):
    cell_area_isf = cell_area.where(mask_front_all == id_closed)
    cell_area_isf_sum = cell_area.where(mask_front_all == id_closed).sum()
    melt_front = melt_front.where(mask_front_all != id_closed, melt_flux.sel(ID=id_closed) * cell_area_isf / cell_area_isf_sum).drop('ID')

Double-check that the total melt is matched

In [ ]:
melt_front_tot_list = []
for id_closed in tqdm(file_Justine_closed.ID):
    melt_front_tot_list.append(melt_front.where(mask_front_all == id_closed).sum().assign_coords({'ID': id_closed}))
melt_front_tot_xr = xr.concat(melt_front_tot_list, dim='ID')

In [ ]:
melt_front.sum()

In [ ]:
melt_front_tot_xr.sum()

In [ ]:
file_Justine_closed['MeltAdusumilli'].sum()

CLEAN AND REMOVE ALL FRONTS WHERE MELT IS ZERO

In [ ]:
zmin_front = zmin_front.where(melt_front > 0)
zmax_front = zmax_front.where(melt_front > 0)
melt_front = melt_front.where(melt_front > 0)

WRITE TO NETCDF

In [ ]:
ds_isfNEMO['z_min'] = zmin_front
ds_isfNEMO['z_min'].attrs['standard_name'] = 'most shallow point of the ice-shelf draft indicated at the ice-shelf front'
ds_isfNEMO['z_min'].attrs['units'] = 'm below sea level'

ds_isfNEMO['z_max'] = zmax_front
ds_isfNEMO['z_max'].attrs['standard_name'] = 'deepest point of the ice-shelf draft indicated at the ice-shelf front'
ds_isfNEMO['z_max'].attrs['units'] = 'm below sea level'

ds_isfNEMO['melt_isf_closed'] = melt_front
ds_isfNEMO['melt_isf_closed'].attrs['standard_name'] = 'melt flux from '+melt_src+' distributed at the ice-shelf front'
ds_isfNEMO['melt_isf_closed'].attrs['units'] = 'Gt/yr'

In [ ]:
ds_isfNEMO

In [ ]:
ds_isfNEMO.attrs=dict(Source='Based on IMBIE2 (Mask_Iceshelf_IMBIE2_v2.nc)',
                      Compatibility='The masked variables are compatible with '+fNEMO_mask,
                      Creator='C. Burgard and P. Mathiot ('+date.today().strftime("%b-%d-%Y")+')')

In [ ]:
ds_isfNEMO.to_netcdf(inputpath_interim + 'masks_for_eORCA1.nc')